In [ ]:
#Importing the libraries
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import model_selection
from sklearn import metrics 
from sklearn import linear_model
from sklearn import ensemble
from sklearn import tree
from sklearn import svm
import xgboost

In [ ]:
#Importing the dataset
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='vYxSqpgXYTE_5CsKrkuRv_pcMjpYBNBmaEy4lu9rgw03',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'exploratoryanalysisofrainfalldata-donotdelete-pr-nsxjcdoqodn5wl'
object_key = 'weatherAUS.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

data = pd.read_csv(body)
data.head()

In [ ]:
#Analyze the data
data.head()

In [ ]:
data.describe()

In [ ]:
data.info()

In [ ]:
data.shape

In [ ]:
data.duplicated().sum()

In [ ]:
data.nunique()

In [ ]:
#target count

data.RainTomorrow.value_counts()

In [ ]:
#Visualization
def plotPerColumnDistribution(df, nGraphShown, nGraphPerRow):
    nunique = df.nunique()
    df = df[[col for col in df if nunique[col] > 1 and nunique[col] < 50]] # For displaying purposes, pick columns that have between 1 and 50 unique values
    nRow, nCol = df.shape
    columnNames = list(df)
    nGraphRow = (nCol + nGraphPerRow - 1) / nGraphPerRow
    plt.figure(num = None, figsize = (6 * nGraphPerRow, 8 * nGraphRow), dpi = 80, facecolor = 'w', edgecolor = 'k')
    for i in range(min(nCol, nGraphShown)):
        plt.subplot(nGraphRow, nGraphPerRow, i + 1)
        columnDf = df.iloc[:, i]
        if (not np.issubdtype(type(columnDf.iloc[0]), np.number)):
            valueCounts = columnDf.value_counts()
            valueCounts.plot.bar()
        else:
            columnDf.hist()
        plt.ylabel('counts')
        plt.xticks(rotation = 90)
        plt.title(f'{columnNames[i]} (column {i})')
    plt.tight_layout(pad = 1.0, w_pad = 1.0, h_pad = 1.0)
    plt.show()

In [ ]:
plotPerColumnDistribution(data, 10, 5)

In [ ]:
plt.figure(figsize=(10,5))
sns.set_style("darkgrid")
plt.title('RainTomorrow value counts')
sns.countplot(x=data["RainTomorrow"]);

In [ ]:
plt.figure(figsize=(14,7))
sns.countplot(x=data["RainToday"], hue=data["RainTomorrow"], palette=sns.color_palette("husl")[4:]);


In [ ]:
plt.figure(figsize=(17,13))
sns.countplot(data=data, y='Location')

plt.title('Location distribution')
plt.xlabel('')
plt.ylabel('')
plt.tight_layout()

In [ ]:
plt.figure(figsize=(17,13))
sns.countplot(data=data, y='Location', hue="RainToday", palette=sns.color_palette("Set2"))

plt.title('Today Rain count by LOC')
plt.xlabel('')
plt.ylabel('')
plt.tight_layout()

In [ ]:
plt.figure(figsize=(10,11))
plt.pie(data["WindDir9am"].value_counts(),
        labels=list(data["WindDir9am"].value_counts().index),
        autopct='%1.2f%%',
        pctdistance=0.8,
       );

In [ ]:
plt.figure(figsize=(10,11))
plt.pie(data["WindDir3pm"].value_counts(),
        labels=list(data["WindDir3pm"].value_counts().index),
        autopct='%1.2f%%',
        pctdistance=0.8,
       );

In [ ]:
data.hist(figsize=(17,13), color="m");

In [ ]:
#Preprocessing

df_preprocessed = data.copy()

In [ ]:
df_preprocessed.isnull().mean() * 100

In [ ]:
mean = df_preprocessed.mean()
df_preprocessed.fillna(mean, inplace=True)

df_preprocessed.isna().sum()

In [ ]:
df_preprocessed.replace(np.nan, 'NaN', inplace=True)
df_preprocessed.isna().sum()

In [ ]:
df_preprocessed.head()

In [ ]:
df_preprocessed['Date'] = pd.to_datetime(df_preprocessed['Date'])

In [ ]:
#columns segregation
date_columns = ['Date']
cat_columns = df_preprocessed.select_dtypes(include=['object']).columns
num_columns = df_preprocessed.select_dtypes(include=['float64', 'int64']).columns
target_col = 'RainTomorrow'

In [ ]:
#Encoding categorical variables
df_preprocessed.head()

In [ ]:
cat_columns

In [ ]:
import numpy as np
from sklearn.impute import SimpleImputer
imp_mode=SimpleImputer(missing_values=np.nan,strategy='most_frequent')

In [ ]:
data_cat = data[['Location','RainToday','WindGustDir','WindDir9am','WindDir3pm','RainTomorrow']]

In [ ]:
#Fitting and Transforming the missing data
data_cat = imp_mode.fit_transform(data_cat)

In [ ]:
for i in cat_columns:
    print('Unique Values in ' + i + ' Column:', end = " ")
    print(len(df_preprocessed[i].unique()))
    print('-----------------------------------------------------')

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for i in ['Location','WindGustDir','WindDir9am','WindDir3pm']:
    df_preprocessed[i] = le.fit_transform(df_preprocessed[i])
df_preprocessed.head()

In [ ]:
df_preprocessed[['RainToday']].value_counts()

In [ ]:
df_preprocessed[['RainTomorrow']].value_counts()

In [ ]:
df_preprocessed[['RainToday','RainTomorrow']].isna().sum()

In [ ]:
dataframe = df_preprocessed[(df_preprocessed != "NaN").all(axis=1)]
del df_preprocessed

In [ ]:
dataframe['RainToday'].value_counts()

In [ ]:
label_dict = {'No': 0,'Yes':1}
dataframe['RainTomorrow'] = dataframe['RainTomorrow'].map(label_dict)
dataframe['RainToday'] = dataframe['RainToday'].map(label_dict)

In [ ]:
dataframe.head()


In [ ]:
# removing Date,Evoporation,Sunshine,Cloud9am,Cloud3pm
dataframe.drop(['Date','Evaporation','Sunshine','Cloud9am','Cloud3pm'], axis=1, inplace=True)

In [ ]:
dataframe.reset_index(drop=True, inplace=True)

In [ ]:
#train test split

from sklearn.model_selection import train_test_split

X = dataframe.drop(columns=["RainTomorrow"])
y = dataframe["RainTomorrow"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
#Feature Scaling
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
X_train_scaled.shape

In [ ]:
X_train_scaled[0,:]

In [ ]:
#Feature Selection
from sklearn.feature_selection import SelectKBest

fs = SelectKBest(k=10)
X_train_scaled = fs.fit_transform(X_train_scaled, y_train)
X_test_scaled = fs.transform(X_test_scaled)

In [ ]:
#Training and testing the model
from sklearn.ensemble import RandomForestClassifier


In [ ]:
from sklearn.svm import SVC

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
#Models initialization of the models
XGBoost = xgboost.XGBRFClassifier()
Rand_forest = RandomForestClassifier()
svm = SVC()
Dtree = DecisionTreeClassifier()
GBM = GradientBoostingClassifier()
log = LogisticRegression()

In [ ]:
#Fitting the model
rand_mod = Rand_forest.fit(X_train_scaled,y_train)

In [ ]:
#Predicting the Train values
p1 = Rand_forest.predict(X_train_scaled)

In [ ]:
#Model Evaluation
#1.Accuracy
t1 = Rand_forest.predict(X_test_scaled)

In [ ]:
print("Rand_forest:",metrics.accuracy_score(y_test,t1))

In [ ]:
#2.Confusion Matrix
#Random forest Confusion matrix
conf_matrix = metrics.confusion_matrix(y_test,t1)

In [ ]:
fig,ax = plt.subplots(figsize=(7.5,7.5))
ax.matshow(conf_matrix,alpha=0.3)
for i in range(conf_matrix.shape[0]):
  for j in range(conf_matrix.shape[1]):
    ax.text(x=j, y=i, s=conf_matrix[i,j], va ='center', ha='center',size='xx-large')
plt.xlabel('Predictions',fontsize=18)
plt.ylabel('Actuals',fontsize=18)
plt.title('Confusion Matrix',fontsize=18)
plt.show()


In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [ ]:
print(conf_matrix)
print("Accuracy:",accuracy_score(y_test,t1))
print("Precision:",precision_score(y_test,t1))
print("Recall:",recall_score(y_test,t1))
print("F1-score:",f1_score(y_test,t1))

In [ ]:
#3.ROC AUC SCORE
auc = metrics.roc_auc_score(y_test,t1)

fpr,tpr,thresholds = metrics.roc_curve(y_test,t1)

plt.figure(figsize = (12,10),dpi=80)
plt.axis('scaled')
plt.xlim([0,1])
plt.ylim([0,1])
plt.title("AUC & ROC Curve")
plt.plot(fpr,tpr,'v')
plt.fill_between(fpr,tpr,facecolor = 'blue',alpha=0.8)
plt.text(1,0.05, 'AUC =%0.4f' % auc, ha='right', fontsize =10, weight ='bold' , color = 'black')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.show()



In [ ]:
!pip install ibm_watson_machine_learning

In [ ]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey" : "PQBr9MBF7mFuSh2VVLfOE-liIA04VH-h5VEk8EfjFIuw"
}

client = APIClient(wml_credentials)

In [ ]:

def guid_from_space_name(client,space_name):
    space = client.spaces.get_details()
    #print(space)
    return(next(item for item in space['resources'] if item['entity']["name"] == space_name)['metadata']['id'])

In [ ]:
space_uid = guid_from_space_name(client,'models')
print("Space UID = " + space_uid)

In [ ]:
client.set.default_space(space_uid)

In [ ]:
client.software_specifications.list()

In [ ]:
software_spec_uid = client.software_specifications.get_uid_by_name("runtime-22.1-py3.9")
software_spec_uid

In [ ]:
model_details = client.repository.store_model(model = rand_mod, meta_props = {
    client.repository.ModelMetaNames.NAME:"rainfall",
    client.repository.ModelMetaNames.TYPE:"scikit-learn_1.0",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID : software_spec_uid
})
model_id = client.repository.get_model_id(model_details)

In [ ]:
model_id

In [ ]:
X_train_scaled[0]

In [ ]:
rand_mod.predict([[5.19157088e-01, 5.39083558e-04, 3.25581395e-01, 1.53846154e-02,
       6.96969697e-01, 6.80000000e-01, 3.20338983e-01, 3.19218241e-01,
       5.19305019e-01, 0.00000000e+00]])